# AnalogDots Shoe Recommendation & Personalized Services

In [62]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

## Load Data

In [63]:
interactions = pd.read_csv("D:/Analogdots_project/New/user_interactions.csv")
users = pd.read_csv("D:/Analogdots_project/New/user_profiles.csv")
shoes = pd.read_csv("D:/Analogdots_project/New/shoe_catalog.csv")

## Prepare Content-Based Vectors

In [64]:
shoes["content"] = shoes["type"] + " " + shoes["material"] + " " + shoes["brand"]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(shoes["content"])
similarity_matrix = cosine_similarity(tfidf_matrix)

## Shoe Recommendation Function

In [65]:
def recommend_shoes(user_id, top_n=5):
    user_history = interactions[
        (interactions.user_id == user_id) & 
        (interactions.interaction_type.isin(["view", "purchase"]))
    ]
    
    if user_history.empty:
        return shoes.sample(top_n)[["shoe_id", "name"]]

    indices = [shoes[shoes.shoe_id == sid].index[0] 
               for sid in user_history["shoe_id"].unique() if sid in shoes.shoe_id.values]

    scores = similarity_matrix[indices].mean(axis=0)
    top_indices = np.argsort(scores)[-top_n:][::-1]
    
    return shoes.iloc[top_indices][["shoe_id", "name"]]


In [66]:
recommend_shoes("U1")

,shoe_id,name
16,S17,Nike Formal
55,S56,Nike Formal
60,S61,Nike Running
6,S7,Adidas Formal
10,S11,Skechers Formal


## Personalized Service Logic

In [67]:
def care_reminder(user_id):
    usage_count = interactions[(interactions.user_id == user_id) & 
                               (interactions.interaction_type == 'view')]["shoe_id"].value_counts()
    reminders = []
    for shoe_id, count in usage_count.items():
        if count > 2:
            care_method = shoes[shoes.shoe_id == shoe_id]['care_requirements'].values[0]
            reminders.append(f"Reminder: It's time to care for shoe {shoe_id} using {care_method}.")
    return reminders

In [68]:
# Define care tips based on shoe category keywords
care_tips = {
    "leather": "Use a leather conditioner once a month and avoid water exposure.",
    "sports": "Air-dry after use and clean with a soft brush.",
    "casual": "Wipe with a damp cloth and avoid machine washing.",
    "formal": "Polish regularly and store in a dust bag.",
    "sneakers": "Use sneaker cleaner and avoid direct sunlight.",
    "boots": "Use waterproof spray and remove dirt after each use."
}


In [69]:
def care_reminder(user_id):
    # Get shoes purchased by the user
    purchased_shoes = interactions[
        (interactions.user_id == user_id) &
        (interactions.interaction_type == "purchase")
    ]["shoe_id"].unique()
    
    if len(purchased_shoes) == 0:
        return ["No purchases found for this user."]

    tips = []
    for shoe_id in purchased_shoes:
        shoe_info = shoes[shoes.shoe_id == shoe_id]
        if shoe_info.empty:
            continue
        desc = shoe_info.iloc[0]["type"].lower()
        
        for keyword, tip in care_tips.items():
            if keyword.lower() in desc:
                tips.append(f"Shoe ID {shoe_id}: {tip}")
                break

    return tips if tips else ["No matching care tips found for user's shoes."]


In [70]:
care_reminder("U1")

['Shoe ID S8: Use waterproof spray and remove dirt after each use.',
 'Shoe ID S17: Use waterproof spray and remove dirt after each use.']

In [71]:
import random
from datetime import datetime, timedelta

def random_past_date(months_ago=6):
    days_ago = random.randint(180, 365)  # between 6 months to 1 year
    return (datetime.now() - timedelta(days=days_ago)).strftime("%Y-%m-%d")

# Apply to your interactions DataFrame
interactions.loc[interactions["interaction_type"] == "purchase", "timestamp"] = \
    [random_past_date() for _ in range(len(interactions[interactions["interaction_type"] == "purchase"]))]


In [72]:
from datetime import datetime, timedelta

def replacement_suggestion(user_id):
    user_purchases = interactions[
        (interactions.user_id == user_id) &
        (interactions.interaction_type == "purchase")
    ]

    if user_purchases.empty:
        return ["No past purchases found for this user."]

    suggestions = []
    current_date = datetime.now()

    for _, row in user_purchases.iterrows():
        shoe_id = row["shoe_id"]
        date_str = row["timestamp"]  # assuming it's a string like "2025-01-01"
        try:
            purchase_date = datetime.strptime(date_str, "%Y-%m-%d")
        except:
            continue  # skip if timestamp format is invalid

        # Get shoe info
        shoe_info = shoes[shoes.shoe_id == shoe_id]
        if shoe_info.empty:
            continue

        price = shoe_info.iloc[0]["price"]

        # Logic: Replace if shoe is older than 6 months or low price
        if (current_date - purchase_date).days > 15 or price < 1500:
            suggestions.append(f"Consider replacing Shoe ID {shoe_id} bought on {date_str} (Price: ₹{price})")

    return suggestions if suggestions else ["No replacement suggestions needed at the moment."]


In [81]:
replacement_suggestion("U16")

['Consider replacing Shoe ID S98 bought on 2025-02-06 (Price: ₹3197.55)',
 'Consider replacing Shoe ID S58 bought on 2025-01-12 (Price: ₹8797.91)',
 'Consider replacing Shoe ID S65 bought on 2024-08-13 (Price: ₹3445.82)']